<a href="https://colab.research.google.com/github/Parikshit07111996/School-Of-AI/blob/master/Assignment_File_Session_5_(4)_Batch_Norm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Target
1. Introducing Batch Norm to increase capacity

# Results
1. Parameters : 11k
2. Best Training Accuracy : 99.92
3. Best Test Accuracy : 99.94

#Analysis
1. We see increase in parameters afer addig Batch Norm
2. And also observe over fitting

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

#Data Transform



In [0]:
# Train Phase Transformation
train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                       ])


# Test Phase Transformation
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                       ])

# Dataset and Creating Train/Test Split

In [3]:
train = datasets.MNIST('./data',train=True, download=True, transform= train_transforms)
test = datasets.MNIST('./data',train=True, download=True, transform= test_transforms)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


#Dataloader Arguments & Test/Train Dataloaders

In [0]:
SEED = 1

cuda = torch.cuda.is_available()

torch.manual_seed(SEED)

if cuda:
  torch.cuda.manual_seed(SEED)

dataloader_args = dict(shuffle=True, batch_size= 128, num_workers=4, pin_memory=True)

train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

test_loader = torch.utils.data.DataLoader(test, **dataloader_args)


#The Model

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Input Block
        self.convblock1 = nn.Sequential(
          nn.Conv2d(1, 10, 3, 1,False),
          nn.BatchNorm2d(10),
          nn.ReLU()     
         ) # Output size = 26
        
        #Convolution Block 1
        self.convblock2 = nn.Sequential(
        nn.Conv2d(10, 10, 3,1,False),
        nn.BatchNorm2d(10),
        nn.ReLU()     
        ) # Output size = 24

        self.convblock3 = nn.Sequential(
        nn.Conv2d(10, 20, 3,1,False),
        nn.BatchNorm2d(20),
        nn.ReLU()     
        ) # Output size = 22

        #Transition Block 1

        self.pool1 = nn.MaxPool2d(2, 2) # Outout_size = 11

        self.convblock4 = nn.Sequential(
            nn.Conv2d(20, 10, 1),
            nn.BatchNorm2d(10),
            nn.ReLU()
        ) # Outout_size = 11

        # Convolution Block 2       
        self.convblock5 = nn.Sequential(
        nn.Conv2d(10, 10, 3),
        nn.BatchNorm2d(10),
        nn.ReLU()    
        ) # output size 9

        self.convblock6 = nn.Sequential(
        nn.Conv2d(10, 20, 3),
        nn.BatchNorm2d(20),
        nn.ReLU()    
        ) # output size 7

        # Output Block 
        self.convblock7 = nn.Sequential(
        nn.Conv2d(20, 10, 1),
        nn.BatchNorm2d(10),
        nn.ReLU()
        )  # output size 7

        self.convblock8 = nn.Sequential(
        nn.Conv2d(10, 10, 7),
        )  # output size 1
        
        
        

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        
        x = x.view(-1, 10)
        return F.log_softmax(x)

#Model Parameters

In [6]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
              ReLU-3           [-1, 10, 26, 26]               0
            Conv2d-4           [-1, 10, 24, 24]             910
       BatchNorm2d-5           [-1, 10, 24, 24]              20
              ReLU-6           [-1, 10, 24, 24]               0
            Conv2d-7           [-1, 20, 22, 22]           1,820
       BatchNorm2d-8           [-1, 20, 22, 22]              40
              ReLU-9           [-1, 20, 22, 22]               0
        MaxPool2d-10           [-1, 20, 11, 11]               0
           Conv2d-11           [-1, 10, 11, 11]             210
      BatchNorm2d-12           [-1, 10, 11, 11]              20
             ReLU-13           [-1, 10, 11, 11]               0
           Conv2d-14             [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


#Training and Testing

In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
      #get samples
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.nll_loss(y_pred, target)
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

#Train & Test Our Model

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 20

for epoch in range(EPOCHS):
  print("EPOCH:", epoch)
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.05481303855776787 Batch_id=468 Accuracy=94.93: 100%|██████████| 469/469 [00:11<00:00, 42.22it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0602, Accuracy: 58930/60000 (98.22%)

EPOCH: 1


loss=0.06648164242506027 Batch_id=468 Accuracy=98.41: 100%|██████████| 469/469 [00:10<00:00, 42.68it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0042, Accuracy: 59952/60000 (99.92%)

EPOCH: 18


loss=0.00601223623380065 Batch_id=468 Accuracy=99.92: 100%|██████████| 469/469 [00:10<00:00, 45.55it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0039, Accuracy: 59962/60000 (99.94%)

EPOCH: 19


loss=0.0014117459068074822 Batch_id=468 Accuracy=99.92: 100%|██████████| 469/469 [00:10<00:00, 45.51it/s]



Test set: Average loss: 0.0036, Accuracy: 59962/60000 (99.94%)

